In [306]:
import pandas as pd
import numpy as np

In [307]:
customers = pd.read_csv("customers.csv")
orders = pd.read_csv("orders.csv")

order_items = pd.read_csv("order_items.csv")

In [308]:
customers["last_name"].str.strip().sort_values()

12     Anderson
113    Anderson
81     Anderson
109    Anderson
4      Anderson
         ...   
68       Wilson
71       Wilson
95       Wilson
40       Wilson
59       Wilson
Name: last_name, Length: 120, dtype: object

In [309]:
orders.sort_values('order_date', ascending = False)

,order_id,customer_id,order_date
213,5214,1075,2024-03-31
383,5384,1111,2024-03-31
215,5216,1103,2024-03-31
207,5208,1023,2024-03-30
14,5015,1019,2024-03-30
...,...,...,...
430,5431,1006,2024-01-01
271,5272,1004,2024-01-01
56,5057,1019,2024-01-01
13,5014,1090,2024-01-01


In [310]:
ny_customers = customers[customers["city"].str.lower().isin(["new york", "nyc"])]
ny_customers.head()
#filtering rows we want to grab

,customer_id,first_name,last_name,city
0,1001,Evan,Smith,New York
3,1004,Aisha,Smith,new york
5,1006,Evan,Patel,new york
7,1008,Maria,Smith,NYC
10,1011,Ben,Davis,New York


In [311]:
recent_orders = orders[orders["order_date"] > "2024-02-14"]
recent_orders.head()

,order_id,customer_id,order_date
1,5002,1045,2024-02-20
5,5006,1031,2024-03-10
7,5008,1061,2024-03-08
9,5010,1054,2024-03-24
12,5013,1074,2024-02-16


In [312]:
high_qty = order_items[order_items["quantity"] >= 2].sort_values("quantity")
high_qty.head()
#464 rows × 4 columns

,order_id,product_name,quantity,unit_price
870,5450,Monitor,2,300
26,5199,Monitor,2,300
22,5047,Mouse,2,25
17,5143,Monitor,2,300
16,5268,USB Cable,2,10


In [313]:
customers["first_name"] = customers["first_name"].str.strip().str.capitalize()
customers["last_name"] = customers["last_name"].str.strip().str.capitalize()


In [314]:
customers.sort_values("last_name")

,customer_id,first_name,last_name,city
12,1013,Priya,Anderson,LA
113,1114,Maria,Anderson,LA
81,1082,Jordan,Anderson,chicago
109,1110,Maria,Anderson,LA
4,1005,Priya,Anderson,LA
...,...,...,...,...
68,1069,Taylor,Wilson,NYC
71,1072,Taylor,Wilson,LA
95,1096,Zoe,Wilson,Los Angeles
40,1041,Evan,Wilson,Los Angeles


In [315]:
customers["city"].value_counts()

city
LA             24
New York       22
chicago        17
Los Angeles    17
new york       15
NYC            15
Chicago        10
Name: count, dtype: int64

In [316]:
city_standardization_mapping = {'LA': 'Los Angeles','NYC': 'New York'}

In [317]:
customers["city"] = customers["city"].replace(city_standardization_mapping)

In [318]:
customers["city"].str.capitalize()
customers["city"] = customers["city"].str.title()

# using.apply()

In [319]:
customers["clean_last_name"] = customers["last_name"].apply(
    lambda x:
    x.strip()
)
# used apply on each value and assigned it to new clean last name column

In [320]:
customers["clean_last_name"].value_counts()

clean_last_name
Davis       19
Smith       18
Patel       17
Brown       16
Chen        16
Anderson    13
Wilson      13
Lopez        8
Name: count, dtype: int64

In [321]:
order_items["total_price"] = order_items["quantity"] * order_items["unit_price"]
order_items

,order_id,product_name,quantity,unit_price,total_price
0,5311,Monitor,3,300,900
1,5303,Keyboard,3,70,210
2,5012,Keyboard,3,70,210
3,5318,Monitor,3,300,900
4,5080,USB Cable,1,10,10
...,...,...,...,...,...
895,5303,USB Cable,2,10,20
896,5117,Monitor,2,300,600
897,5124,Keyboard,1,70,70
898,5399,USB Cable,2,10,20


In [322]:
order_items["total_price_applied"] = order_items.apply(
    lambda row:
    row["quantity"] * row["unit_price"], axis = 1
)
order_items
#here we are feeding entire dataset 

,order_id,product_name,quantity,unit_price,total_price,total_price_applied
0,5311,Monitor,3,300,900,900
1,5303,Keyboard,3,70,210,210
2,5012,Keyboard,3,70,210,210
3,5318,Monitor,3,300,900,900
4,5080,USB Cable,1,10,10,10
...,...,...,...,...,...,...
895,5303,USB Cable,2,10,20,20
896,5117,Monitor,2,300,600,600
897,5124,Keyboard,1,70,70,70
898,5399,USB Cable,2,10,20,20


In [323]:
order_items["order_type"] = order_items.apply(
    lambda row:(
        "Large Expensive Order" if row["quantity"] >=3 and row["unit_price"] >50
        else "Bulk Order" if row["quantity"] >=3
        else "Standard Order"
    ), axis =1
)
order_items
# for each row pass the row in the function we provided

,order_id,product_name,quantity,unit_price,total_price,total_price_applied,order_type
0,5311,Monitor,3,300,900,900,Large Expensive Order
1,5303,Keyboard,3,70,210,210,Large Expensive Order
2,5012,Keyboard,3,70,210,210,Large Expensive Order
3,5318,Monitor,3,300,900,900,Large Expensive Order
4,5080,USB Cable,1,10,10,10,Standard Order
...,...,...,...,...,...,...,...
895,5303,USB Cable,2,10,20,20,Standard Order
896,5117,Monitor,2,300,600,600,Standard Order
897,5124,Keyboard,1,70,70,70,Standard Order
898,5399,USB Cable,2,10,20,20,Standard Order


# Merging Tables

In [324]:
orders_with_customers = orders.merge(customers, how = "inner")
orders_with_customers.head()

,order_id,customer_id,order_date,first_name,last_name,city,clean_last_name
0,5001,1089,2024-01-26,Aisha,Smith,New York,Smith
1,5002,1045,2024-02-20,Priya,Davis,New York,Davis
2,5003,1004,2024-02-14,Aisha,Smith,New York,Smith
3,5004,1036,2024-02-13,Chloe,Davis,New York,Davis
4,5005,1070,2024-01-05,Jordan,Brown,Los Angeles,Brown


In [325]:
orders_customers_items = orders_with_customers.merge(order_items, on = "order_id", how ="inner")
orders_customers_items.head()

,order_id,customer_id,order_date,first_name,last_name,city,clean_last_name,product_name,quantity,unit_price,total_price,total_price_applied,order_type
0,5001,1089,2024-01-26,Aisha,Smith,New York,Smith,Monitor,1,300,300,300,Standard Order
1,5001,1089,2024-01-26,Aisha,Smith,New York,Smith,Desk Lamp,2,45,90,90,Standard Order
2,5001,1089,2024-01-26,Aisha,Smith,New York,Smith,Desk Lamp,1,45,45,45,Standard Order
3,5002,1045,2024-02-20,Priya,Davis,New York,Davis,Monitor,2,300,600,600,Standard Order
4,5002,1045,2024-02-20,Priya,Davis,New York,Davis,Monitor,2,300,600,600,Standard Order


In [326]:
combined = (
    orders.merge(customers, on ="customer_id", how ="inner").merge(order_items, on = "order_id", how = "inner")
)
combined

,order_id,customer_id,order_date,first_name,last_name,city,clean_last_name,product_name,quantity,unit_price,total_price,total_price_applied,order_type
0,5001,1089,2024-01-26,Aisha,Smith,New York,Smith,Monitor,1,300,300,300,Standard Order
1,5001,1089,2024-01-26,Aisha,Smith,New York,Smith,Desk Lamp,2,45,90,90,Standard Order
2,5001,1089,2024-01-26,Aisha,Smith,New York,Smith,Desk Lamp,1,45,45,45,Standard Order
3,5002,1045,2024-02-20,Priya,Davis,New York,Davis,Monitor,2,300,600,600,Standard Order
4,5002,1045,2024-02-20,Priya,Davis,New York,Davis,Monitor,2,300,600,600,Standard Order
...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,5450,1008,2024-02-26,Maria,Smith,New York,Smith,Laptop,2,1200,2400,2400,Standard Order
896,5450,1008,2024-02-26,Maria,Smith,New York,Smith,Keyboard,3,70,210,210,Large Expensive Order
897,5450,1008,2024-02-26,Maria,Smith,New York,Smith,USB Cable,2,10,20,20,Standard Order
898,5450,1008,2024-02-26,Maria,Smith,New York,Smith,Mouse,3,25,75,75,Bulk Order


# Grouping and Aggregation

In [327]:
order_items["total_price"].sum()

np.int64(420615)

In [328]:
order_items.groupby("product_name")["total_price"].sum().sort_values(ascending=False)

product_name
Laptop       296400
Monitor       90000
Keyboard      14140
Desk Lamp     10755
Mouse          6750
USB Cable      2570
Name: total_price, dtype: int64

In [329]:
order_items.groupby("product_name")["quantity"].sum().sort_values(ascending=False)

product_name
Monitor      300
Mouse        270
USB Cable    257
Laptop       247
Desk Lamp    239
Keyboard     202
Name: quantity, dtype: int64

In [330]:
order_items.groupby("product_name").agg(
    total_revenue = ("total_price", "sum"),
    avg_quantity = ("quantity","mean")
)

,total_revenue,avg_quantity
product_name,,
Desk Lamp,10755,1.593333
Keyboard,14140,1.578125
Laptop,296400,1.776978
Monitor,90000,1.754386
Mouse,6750,1.719745
USB Cable,2570,1.658065


In [331]:
combined.groupby("customer_id")["total_price"].sum().sort_values(ascending=False).head()

customer_id
1008    13625
1057    13330
1087    11170
1061    10645
1070     9945
Name: total_price, dtype: int64

# Reshaping Data

In [332]:
sample = combined[["customer_id", "product_name","total_price"]].head(12)
sample

,customer_id,product_name,total_price
0,1089,Monitor,300
1,1089,Desk Lamp,90
2,1089,Desk Lamp,45
3,1045,Monitor,600
4,1045,Monitor,600
5,1004,Keyboard,140
6,1004,Desk Lamp,135
7,1004,Laptop,2400
8,1004,USB Cable,20
9,1036,Mouse,50


In [ ]:
pivot = sample.pivot_table(index = "customer_id", columns ="product_name", values = "total_price", aggfunc = "sum")
pivot
#pivot = sample.pivot_table(index="customer_id", columns = "product_name", values="total_price", aggfunc="sum")

product_name,Desk Lamp,Keyboard,Laptop,Monitor,Mouse,USB Cable
customer_id,,,,,,
1004,135.0,140.0,2400.0,NaN,NaN,20.0
1036,NaN,NaN,NaN,NaN,50.0,20.0
1045,NaN,NaN,NaN,1200.0,NaN,NaN
1089,135.0,NaN,NaN,300.0,NaN,NaN


In [334]:
pivot.reset_index().melt(
    id_vars = "customer_id",
    var_name = "product_name",
    value_name = "revenue_name"
)

,customer_id,product_name,revenue_name
0,1004,Desk Lamp,135.0
1,1036,Desk Lamp,NaN
2,1045,Desk Lamp,NaN
3,1089,Desk Lamp,135.0
4,1004,Keyboard,140.0
5,1036,Keyboard,NaN
6,1045,Keyboard,NaN
7,1089,Keyboard,NaN
8,1004,Laptop,2400.0
9,1036,Laptop,NaN
